# Real Business Cycle estimation 

Taken from http://www.chadfulton.com/topics/estimating_rbc.html

Here, we are going to estimate a Real Business Cycle DSGE Model by Maximum Likelihood in Python

In [1]:
#import packages
%matplotlib inline
from __future__ import division

import numpy as np
from scipy import optimize, signal
import pandas as pd
#from pandas_datareader.data import DataReader
import statsmodels.api as sm
from statsmodels.tools.numdiff import approx_fprime, approx_fprime_cs
from IPython.display import display
import matplotlib.pyplot as plt
# import seaborn as sn

from numpy.testing import assert_allclose

# Set some pretty-printing options
np.set_printoptions(precision=3, suppress=True, linewidth=120)
pd.set_option('float_format', lambda x: '%.3g' % x, )

## Household Problem

$$
\max E_0 \sum_{t=0}^\infty \beta^t u(c_t, l_t)
$$
subject to:

the budget constraint: 
$$ yt=ct+it $$
the capital accumulation equation: $$ kt+1=(1−δ)kt+it $$
$$ 1=lt+nt $$
where households have the following production technology:
$$
y_t = z_t f(k_t, n_t)
$$
and where the (log of the) technology process follows an AR(1) process:
$$
\log z_t = \rho \log z_{t-1} + \varepsilon_t, \qquad \varepsilon_t \sim N(0, \sigma^2)
$$

## Functional forms

We will assume additively separable utility in consumption and leisure, with:

$$u(c_t, n_t) = \log(c_t) + \psi l_t$$
where $\psi$ is a scalar weight.

The production function will be Cobb-Douglas:

$$
f(k_t, n_t) = k_t^\alpha n_t^{1 - \alpha}
$$

## Optimal behavior

Optimal household behavior is standard:

$
\psi = \frac{1}{c_{t}}(1-\alpha)z_{t}  (\frac{k_{t}}{n_{t}})^\alpha
$

$
\frac{1}{c_{t}} = \beta E_{t}  \left\{\frac{1}{c_{t+1}} \left[\alpha z_{t+1} \left(\frac{k_{t+1}}{n_{t+1}}\right)^ {\alpha-1} + (1 - \delta)\right]\right\}
$



## Nonlinear system

$ \psi c_{t} = (1-\alpha) z_{t} (\frac{k_{t}}{n_{t}})^ \alpha $,      Static FOC

$ \frac{1}{c_{t}} = \beta E_{t}  \left\{\frac{1}{c_{t+1}} \left[\alpha z_{t+1} \left(\frac{k_{t+1}}{n_{t+1}}\right)^ {\alpha-1} + (1 - \delta)\right]\right\}
$, Euler equation

$ y_{t} = z_{t} k_{t}^\alpha n_{t}^{1-\alpha}$,  Production function

$ y_{t} = c_{t} + i_{t}$,  Aggregate resource constrainte

$ k_{t+1} = (1-\delta) k_{t} + i_{t }$ ,  Capital accumulation

$ 1 = l_{t} + n_{t} $ , Labor-Leisure Trade-off

$ log z_{t} = \rho log z_{t-1} + \varepsilon_{t} $ , Technology shock transition



## Variables / Parameters

This system is in the following variables:


$x_{t} = \{ \\
y_{t}, \text{Output} \\
c_{t}, \text{Consumption} \\
i_{t}, \text{Investment} \\
n_{t}, \text{Labor} \\
l_{t}, \text{Leisure} \\
k_{t}, \text{Capital} \\
z_{t}, \text{Technology} \\
            \\
\}$

and depends on the following parameters:
$ \{ \\
\beta, \text{Discount rate} \\
\psi, \text{Marginal disutility of labor} \\
\delta, \text{Depreciation rate} \\
\alpha, \text{Capital-share output} \\
\rho, \text{Technology shock persistence} \\
\sigma^2, \text{Technology shock variance} \\
            \\
\}$

In [2]:
# Save the names of the equations, variables, and parameters
equation_names = [
    'static FOC', 'euler equation', 'production',
    'aggregate resource constraint', 'capital accumulation',
    'labor-leisure', 'technology shock transition'
]
variable_names = [
    'output', 'consumption', 'investment',
    'labor', 'leisure', 'capital', 'technology'
]
parameter_names = [
    'discount rate', 'marginal disutility of labor',
    'depreciation rate', 'capital share',
    'technology shock persistence',
    'technology shock standard deviation',
]

# Save some symbolic forms for pretty-printing
variable_symbols = [
    r"y", r"c", r"i", r"n", r"l", r"k", r"z",
]
contemporaneous_variable_symbols = [
    r"$%s_t$" % symbol for symbol in variable_symbols
]
lead_variable_symbols = [
    r"$%s_{t+1}$" % symbol for symbol in variable_symbols
]

parameter_symbols = [
    r"$\beta$", r"$\psi$", r"$\delta$", r"$\alpha$", r"$\rho$", r"$\sigma^2$"
]

## Numerical method

Using these equations, we can numerically find the steady-state using a root-finding algorithm and can then log-linearize around the steady-state using a numerical gradient procedure. In particular, here we follow DeJong and Dave (2011) chapter 2 and 3.1.

Write the generic non-linear system as:

$\Gamma(E_t x_{t+1}, x_t, z_t) = 0$
in the absense of shocks, this can be rewritten as:

$\Psi(x_{t+1}, x_t) = 0$
or as:

$\Psi_1(x_{t+1}, x_t) = \Psi_2(x_{t+1}, x_t)$
and finally in logs as

$\log \Psi_1(e^{\log x_{t+1}}, e^{\log x_t}) - \log \Psi_2(e^{\log x_{t+1}}, e^{\log x_t}) = 0$
First, we define a new class (```RBC1```) which holds the state of the RBC model (its dimensions, parameters, etc) and has methods for evaluating the log system. The ```eval_logged``` method evaluates the last equation, above.

Notice that the order of variables and order of equations is as described above.

In [3]:
class RBC1(object):
    def __init__(self, params=None):
        # Model dimensions
        self.k_params = 6
        self.k_variables = 7
        
        # Initialize parameters
        if params is not None:
            self.update(params)
    
    def update(self, params):
        # Save deep parameters
        self.discount_rate = params[0]
        self.disutility_labor = params[1]
        self.depreciation_rate = params[2]
        self.capital_share = params[3]
        self.technology_shock_persistence = params[4]
        self.technology_shock_std = params[5]
        
    def eval_logged(self, log_lead, log_contemporaneous):
        (log_lead_output, log_lead_consumption, log_lead_investment,
         log_lead_labor, log_lead_leisure, log_lead_capital,
         log_lead_technology_shock) = log_lead
        
        (log_output, log_consumption, log_investment, log_labor,
         log_leisure, log_capital, log_technology_shock) = log_contemporaneous
        
        return np.r_[
            self.log_static_foc(
                log_lead_consumption, log_lead_labor,
                log_lead_capital, log_lead_technology_shock
            ),
            self.log_euler_equation(
                log_lead_consumption, log_lead_labor,
                log_lead_capital, log_lead_technology_shock,
                log_consumption
            ),
            self.log_production(
                log_lead_output, log_lead_labor, log_lead_capital,
                log_lead_technology_shock
            ),
            self.log_aggregate_resource_constraint(
                log_lead_output, log_lead_consumption,
                log_lead_investment
            ),
            self.log_capital_accumulation(
                log_lead_capital, log_investment, log_capital
            ),
            self.log_labor_leisure_constraint(
                log_lead_labor, log_lead_leisure
            ),
            self.log_technology_shock_transition(
                log_lead_technology_shock, log_technology_shock
            )
        ]
    
    def log_static_foc(self, log_lead_consumption, log_lead_labor,
                       log_lead_capital, log_lead_technology_shock):
        return (
            np.log(self.disutility_labor) +
            log_lead_consumption -
            np.log(1 - self.capital_share) -
            log_lead_technology_shock -
            self.capital_share * (log_lead_capital - log_lead_labor)
        )
        
    def log_euler_equation(self, log_lead_consumption, log_lead_labor,
                           log_lead_capital, log_lead_technology_shock,
                           log_consumption):
        return (
            -log_consumption -
            np.log(self.discount_rate) +
            log_lead_consumption -
            np.log(
                (self.capital_share *
                 np.exp(log_lead_technology_shock) * 
                 np.exp((1 - self.capital_share) * log_lead_labor) /
                 np.exp((1 - self.capital_share) * log_lead_capital)) +
                (1 - self.depreciation_rate)
            )
        )
        
    def log_production(self, log_lead_output, log_lead_labor, log_lead_capital,
                       log_lead_technology_shock):
        return (
            log_lead_output -
            log_lead_technology_shock -
            self.capital_share * log_lead_capital -
            (1 - self.capital_share) * log_lead_labor
        )
        
    def log_aggregate_resource_constraint(self, log_lead_output, log_lead_consumption,
                                          log_lead_investment):
        return (
            log_lead_output -
            np.log(np.exp(log_lead_consumption) + np.exp(log_lead_investment))
        )
    
    def log_capital_accumulation(self, log_lead_capital, log_investment, log_capital):
        return (
            log_lead_capital -
            np.log(np.exp(log_investment) + (1 - self.depreciation_rate) * np.exp(log_capital))
        )
    
    def log_labor_leisure_constraint(self, log_lead_labor, log_lead_leisure):
        return (
            -np.log(np.exp(log_lead_labor) + np.exp(log_lead_leisure))
        )
    
    def log_technology_shock_transition(self, log_lead_technology_shock, log_technology_shock):
        return (
            log_lead_technology_shock -
            self.technology_shock_persistence * log_technology_shock
        )

Later we will estimate (some of) the parameters; in the meantime we fix them at the values used to generate the datasets in Ruge-Murcia (2007).

In [4]:
# Setup fixed parameters
parameters = pd.DataFrame({
    'name': parameter_names,
    'value': [0.95, 3, 0.025, 0.36, 0.85, 0.04]
})
parameters.T

,0,1,2,3,4,5
name,discount rate,marginal disutility of labor,depreciation rate,capital share,technology shock persistence,technology shock standard deviation
value,0.95,3,0.025,0.36,0.85,0.04


## Steady State

### Numeric calculation
To numerically calculate steady-state, we apply a root-finding algorithm to the ```eval_logged``` method. In particular, we are finding values $\bar x$ such that

$log \Psi_1(e^{\log \bar x}, e^{\log \bar x}) - \log \Psi_2(e^{\log \bar x}, e^{\log \bar x}) = 0 $
These will be confirmed analytically, below.

Here we create a derived class, ```RBC2``` which extends all of the functionality from above, but now includes methods for numerical calcualtion of the steady-state.

In [5]:
class RBC2(RBC1):
    def steady_state_numeric(self):
        # Setup starting parameters
        log_start_vars = [0.5] * self.k_variables  # very arbitrary

        # Setup the function the evaluate
        eval_logged = lambda log_vars: self.eval_logged(log_vars, log_vars)

        # Apply the root-finding algorithm
        result = optimize.root(eval_logged, log_start_vars)
        
        return np.exp(result.x)

mod2 = RBC2(parameters['value'])

steady_state = pd.DataFrame({
    'value': mod2.steady_state_numeric()
}, index=variable_names)

steady_state.T

,output,consumption,investment,labor,leisure,capital,technology
value,0.572,0.506,0.0663,0.241,0.759,2.65,1


### Analytic evaluation
In this case, we can analytically evaluate the steady-state:
$$
\frac{\bar y}{\bar n} = n \\
\frac{\bar c}{\bar n} = n - \delta \theta \\
\frac{\bar i}{\bar n} = \delta \theta \\
\bar n = \frac{1}{(\frac{1}{1-\alpha})\psi [1 - \delta \theta^{1-\alpha}]} \\
\bar l = 1 - \bar n \\
\frac{\bar k}{\bar n} = \theta
$$
where
$$
\theta = \left(\frac{\alpha}{1/\beta - (1- \delta)}\right)^\frac{1}{1-\alpha} \\
\eta = \theta^\alpha
$$

In [6]:
class RBC3(RBC2):
    
    def update(self, params):
        # Update the deep parameters
        super(RBC3, self).update(params)
        
        # And now also calculate some intermediate parameters
        self.theta = (self.capital_share / (
            1 / self.discount_rate -
            (1 - self.depreciation_rate)
        ))**(1 / (1 - self.capital_share))
        
        self.eta = self.theta**self.capital_share
    
    def steady_state_analytic(self):
        steady_state = np.zeros(7)

        # Labor (must be computed first)
        numer = (1 - self.capital_share) / self.disutility_labor
        denom = (1 - self.depreciation_rate * self.theta**(1 - self.capital_share))
        steady_state[3] = numer / denom
        # Output
        steady_state[0] = self.eta * steady_state[3]
        # Consumption
        steady_state[1] = (1 - self.capital_share) * self.eta / self.disutility_labor
        # Investment
        steady_state[2] = self.depreciation_rate * self.theta * steady_state[3]
        # Labor (computed already)
        # Leisure
        steady_state[4] = 1 - steady_state[3]
        # Capital
        steady_state[5] = self.theta * steady_state[3]
        # Technology shock
        steady_state[6] = 1
        
        return steady_state
    
mod3 = RBC3(parameters['value'])

steady_state = pd.DataFrame({
    'numeric': mod3.steady_state_numeric(),
    'analytic': mod3.steady_state_analytic()
}, index=variable_names)

steady_state.T

,output,consumption,investment,labor,leisure,capital,technology
numeric,0.572,0.506,0.0663,0.241,0.759,2.65,1
analytic,0.572,0.506,0.0663,0.241,0.759,2.65,1


## Log-linearization

The system we wrote down, above, was non-linear. In order to estimate it, we want to get it in a linear form:

$$A E_t \tilde x_{t+1} = B \tilde x_{t} + C v_{t+1}$$
where $v_{t+1}$ contains structural shocks (here, $z_{t}$ is included in the $x_{t}$ vector, and the only structural shock is $\varepsilon_{t}$, the innovation to $z_{t}$).

This can be achieved via log-linearization around the steady state. In this case, DeJong and Dave (2011) show that:

$$A = \left [ \frac{\partial \log [ \Psi_1 ]}{\partial \log(x_{t+1})} (\bar x) - \frac{\partial \log [ \Psi_2 ]}{\partial \log(x_{t+1})} (\bar x) \right ] \\ B = - \left [ \frac{\partial \log [ \Psi_1 ]}{\partial \log(x_t)} (\bar x) - \frac{\partial \log [ \Psi_2 ]}{\partial \log(x_t)} (\bar x) \right ] \\
$$
where $\bar x_{t} = log\left( \frac{x_{t}}{\bar x}\right)$ expresses the variables in proportional deviation from steady-state form.

The matrix $C$ can be constructed by observation. In this case, we have:
$$
\begin{bmatrix}
0 & 0 & 0 &  0 &  0 &  0 &  1
\end{bmatrix} \\
\qquad \text{and} \qquad v_{t+1} \equiv \varepsilon_t
$$

### Numeric calculation
Since the ```eval_logged``` method of our class evaluates 
$ log \psi_{1}(e^{log x_{t+1}}, e^{log x_{t}} )$ we can apply a numerical gradient procedure to it to get $A$, when we differentiate with respect to the lead variables, and $B$, when we differentiate with respect to the contemporaneous variables.

In [7]:
class RBC4(RBC3):
    
    def A_numeric(self):
        log_steady_state = np.log(self.steady_state_analytic())

        eval_logged_lead = lambda log_lead: self.eval_logged(log_lead, log_steady_state)
        
        return approx_fprime_cs(log_steady_state, eval_logged_lead)

    def B_numeric(self):
        log_steady_state = np.log(self.steady_state_analytic())
        
        eval_logged_contemporaneous = lambda log_contemp: self.eval_logged(log_steady_state, log_contemp)
        
        return -approx_fprime_cs(log_steady_state, eval_logged_contemporaneous)
    
    def C(self):
        return np.r_[[0]*(self.k_variables-1), 1]

mod4 = RBC4(parameters['value'])
        
display(pd.DataFrame(mod4.A_numeric(), index=equation_names, columns=lead_variable_symbols))
display(pd.DataFrame(mod4.B_numeric(), index=equation_names, columns=contemporaneous_variable_symbols))
display(pd.DataFrame(mod4.C(), index=equation_names, columns=[r'$\varepsilon_t$']))

,$y_{t+1}$,$c_{t+1}$,$i_{t+1}$,$n_{t+1}$,$l_{t+1}$,$k_{t+1}$,$z_{t+1}$
static FOC,0,1,0,0.36,0,-0.36,-1
euler equation,0,1,0,-0.0472,0,0.0472,-0.0737
production,1,0,0,-0.64,0,-0.36,-1
aggregate resource constraint,1,-0.884,-0.116,0,0,0,0
capital accumulation,0,0,0,0,0,1,0
labor-leisure,-0,-0,-0,-0.241,-0.759,-0,-0
technology shock transition,0,0,0,0,0,0,1


,$y_t$,$c_t$,$i_t$,$n_t$,$l_t$,$k_t$,$z_t$
static FOC,-0,-0,-0,-0,-0,-0,-0
euler equation,-0,1,-0,-0,-0,-0,-0
production,-0,-0,-0,-0,-0,-0,-0
aggregate resource constraint,-0,-0,-0,-0,-0,-0,-0
capital accumulation,-0,-0,0.025,-0,-0,0.975,-0
labor-leisure,-0,-0,-0,-0,-0,-0,-0
technology shock transition,-0,-0,-0,-0,-0,-0,0.85


,$\varepsilon_t$
static FOC,0
euler equation,0
production,0
aggregate resource constraint,0
capital accumulation,0
labor-leisure,0
technology shock transition,1


### Analytic Evaluation
This system can be log-linearized directly, as well, yielding (see e.g. Ruge-Murcia (2007) Appendix A for these formulas, with slightly different notation):
$$
\tilde c_{t} = \tilde z_{t} + \alpha \tilde k_{t} - \alpha \tilde n_{t} \\
-\tilde c_{t} = E_{t} \tilde c_{t+1} + \zeta (\alpha - 1) E_{t} \tilde k_{t+1} + \zeta(1-\alpha) E_{t} \tilde n_{t+1} + \zeta E \tilde z_{t+1} \\
\tilde y_{t} = \tilde z_{t} + \alpha \tilde k_{t} + (1- \alpha) \tilde n_{t} \\
\tilde y_{t} = \gamma \tilde c_{t} + (1-\gamma) \tilde i_{t} \\
\tilde k_{t+1} = (1-\delta)\tilde k_{t} + \delta \tilde i_{t} \\
0 = \frac{\bar l}{\bar l + \bar n} \tilde l_{t+1} + \frac{\bar n}{\bar l + \bar n}\tilde n_{t+1} \\
\tilde z_{t+1} = \rho \tilde z_{t} + \varepsilon_{t}
$$


where $\zeta = \alpha \beta \theta^{\alpha - 1}$ and $ \gamma = 1 - \delta \theta ^{1-\alpha}$ is the steady-state consumption-output ratio.

In [8]:
class RBC5(RBC4):
    
    def update(self, params):
        super(RBC5, self).update(params)
        
        # Now calculate some more intermediate parameters
        self.gamma = 1 - self.depreciation_rate * self.theta**(1 - self.capital_share)
        self.zeta = self.capital_share * self.discount_rate * self.theta**(self.capital_share - 1)
    
    def A_analytic(self):
        steady_state = self.steady_state_analytic()
        
        A = np.array([
            [0, 1, 0, self.capital_share, 0, -self.capital_share, -1],
            [0, 1, 0, self.zeta * (self.capital_share - 1), 0, self.zeta * (1 - self.capital_share), -self.zeta],
            [1, 0, 0, (self.capital_share - 1), 0, -self.capital_share, -1],
            [1, -self.gamma, (self.gamma - 1), 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, -steady_state[3], -steady_state[4], 0, 0],
            [0, 0, 0, 0, 0, 0, 1],
        ])
        
        return A

    def B_analytic(self):
        
        B = np.array([
            [0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, self.depreciation_rate, 0, 0, 1 - self.depreciation_rate, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, self.technology_shock_persistence],
        ])
        
        return B

mod5 = RBC5(parameters['value'])

display(pd.DataFrame(mod5.A_analytic(), index=equation_names, columns=lead_variable_symbols))
assert(np.all(np.abs(mod5.A_numeric() - mod5.A_analytic()) < 1e-10))

display(pd.DataFrame(mod5.B_analytic(), index=equation_names, columns=lead_variable_symbols))
assert(np.all(np.abs(mod5.B_numeric() - mod5.B_analytic()) < 1e-10))

,$y_{t+1}$,$c_{t+1}$,$i_{t+1}$,$n_{t+1}$,$l_{t+1}$,$k_{t+1}$,$z_{t+1}$
static FOC,0,1,0,0.36,0,-0.36,-1
euler equation,0,1,0,-0.0472,0,0.0472,-0.0737
production,1,0,0,-0.64,0,-0.36,-1
aggregate resource constraint,1,-0.884,-0.116,0,0,0,0
capital accumulation,0,0,0,0,0,1,0
labor-leisure,0,0,0,-0.241,-0.759,0,0
technology shock transition,0,0,0,0,0,0,1


,$y_{t+1}$,$c_{t+1}$,$i_{t+1}$,$n_{t+1}$,$l_{t+1}$,$k_{t+1}$,$z_{t+1}$
static FOC,0,0,0,0,0,0,0
euler equation,0,1,0,0,0,0,0
production,0,0,0,0,0,0,0
aggregate resource constraint,0,0,0,0,0,0,0
capital accumulation,0,0,0.025,0,0,0.975,0
labor-leisure,0,0,0,0,0,0,0
technology shock transition,0,0,0,0,0,0,0.85


### System Reduction
The system currently has 7 equations in 7 unknowns. This can be reduced into a system of 3 equations in 3 unknowns (consumption, capital, and the technology shock), by substituting out output, investment, labor, and leisure. Given a solution to the reduced system, the remaining four unknowns can be calculated. Also, we remove the technology shock transition from the state vector and include it instead in the stochastic shocks component $(v_{t+1})$.

First, notice that (using the static first order condition, production function, aggregate resource constraint, and labor-leisure tradeoff equations, respectively):
$$
\tilde n_{t} = \frac{1}{\alpha} \tilde c_{t} + \frac{1}{\alpha} \tilde z_{t} + \tilde k_{t}\\
\tilde y_{t} = \tilde z_{t} + \alpha \tilde k_{t} + (1-\alpha) \left[ - \frac{1}{\alpha} \tilde c_{t} + \frac{1}{\alpha} \tilde z_{t} + \tilde k_{t}\right] \\
 = \frac{1}{\alpha} \tilde z_{t} - \frac{1-\alpha}{\alpha} \tilde c_{t} + \tilde k_{t} \\
\tilde i_{t} = \frac{1}{1-\gamma} \left[ \frac{1}{\alpha} \tilde z_{t} - \frac{1-\alpha}{\alpha}\tilde c_{t} + \tilde k_{t} - \gamma \tilde c_{t} \right] \\
= \frac{1}{1-\gamma}  \frac{1}{\alpha} \tilde z_{t} - \frac{1 - \alpha + \gamma \alpha}{\alpha} \tilde c_{t} + \tilde k_{t}\\
\tilde l_{t} = - \frac{\bar n}{1 - \bar n} \tilde n_{t}
$$

then we can plug these values into the remaining three equations (Euler, capital accumulation, and shock transition equations):
$$

$$
